### Descarga el último archivo gerenado por el sensor GLM del GOES-19 del repositorio

In [1]:
# Instala las librerías necesarias
!pip install netCDF4
!pip install datetime
!pip install glob
!pip install s3fs


[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement glob (from versions: none)
ERROR: No matching distribution found for glob

[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
# Importa las librerías necesarias
import os
import numpy as np
from netCDF4 import Dataset
import netCDF4 as nc
import s3fs
import datetime
import glob

In [3]:
# Obtiene fecha y hora actual UTC
time_utc = datetime.datetime.utcnow()
año_tex = str(time_utc.year)
hora_tex = f"{time_utc.hour:02d}"  # Dos dígitos
dia_tex = f"{time_utc.timetuple().tm_yday:03d}"  # Tres dígitos

In [4]:
print(año_tex)
print(dia_tex)
print(hora_tex)

2025
255
23


In [5]:
# Rutas de archivos
ruta_notebook = os.getcwd()
ruta_files = os.path.join(ruta_notebook, 'Archivos')

In [6]:
# Crea la carpeta Archivos si no existe
os.makedirs(ruta_files, exist_ok=True)

In [7]:
# Limpia los archivos .nc ya existentes en Archivos
files_existentes = glob.glob(os.path.join(ruta_files, '*.nc'))
for f in files_existentes:
    os.remove(f)

In [8]:
# Conecta con el bucket público de GOES-19
fs = s3fs.S3FileSystem(anon=True)

In [9]:
# función para buscar 12 archivos hacia atrás
def obtener_archivos(fs, año_tex, dia_tex, hora_tex, max_archivos=12):
    archivos = []

    # Convertir a datetime para retroceder horas fácilmente
    dt = datetime.datetime.strptime(f"{año_tex}{dia_tex}{hora_tex}", "%Y%j%H")

    while len(archivos) < max_archivos:
        # Construir ruta S3 para año, día juliano y hora
        ruta_s3 = f'noaa-goes19/GLM-L2-LCFA/{dt.year}/{dt.timetuple().tm_yday:03d}/{dt.hour:02d}/'

        try:
            files_s3 = np.array(fs.ls(ruta_s3))
            if len(files_s3) > 0:
                archivos.extend(list(files_s3))
        except FileNotFoundError:
            # Si no existe la ruta, seguimos retrocediendo
            pass

        # Si ya tenemos suficientes, cortamos
        if len(archivos) >= max_archivos:
            break

        # Retrocedemos una hora
        dt -= datetime.timedelta(hours=1)

    # Nos quedamos con los últimos "max_archivos", ordenados
    archivos = sorted(archivos)[-max_archivos:]
    return archivos

In [10]:
# Uso de la función
archivos_descargar = obtener_archivos(fs, año_tex, dia_tex, hora_tex, max_archivos=12)

if len(archivos_descargar) == 0:
    print("No se encontraron archivos en las últimas horas.")
else:
    for archivo in archivos_descargar:
        nombre_archivo = archivo.split('/')[-1]
        destino = os.path.join(ruta_files, nombre_archivo)

        # Descarga el archivo a la carpeta Archivos
        fs.get(archivo, destino)
        print(f"Archivo descargado: {destino}")

Archivo descargado: c:\Users\fabri\Documents\Python\Desarrollos\Storm_Detector\GOES_19\Scripts\Descarga\Archivos\OR_GLM-L2-LCFA_G19_s20252552331000_e20252552331200_c20252552331217.nc
Archivo descargado: c:\Users\fabri\Documents\Python\Desarrollos\Storm_Detector\GOES_19\Scripts\Descarga\Archivos\OR_GLM-L2-LCFA_G19_s20252552331200_e20252552331400_c20252552331419.nc
Archivo descargado: c:\Users\fabri\Documents\Python\Desarrollos\Storm_Detector\GOES_19\Scripts\Descarga\Archivos\OR_GLM-L2-LCFA_G19_s20252552331400_e20252552332000_c20252552332016.nc
Archivo descargado: c:\Users\fabri\Documents\Python\Desarrollos\Storm_Detector\GOES_19\Scripts\Descarga\Archivos\OR_GLM-L2-LCFA_G19_s20252552332000_e20252552332200_c20252552332217.nc
Archivo descargado: c:\Users\fabri\Documents\Python\Desarrollos\Storm_Detector\GOES_19\Scripts\Descarga\Archivos\OR_GLM-L2-LCFA_G19_s20252552332200_e20252552332400_c20252552332418.nc
Archivo descargado: c:\Users\fabri\Documents\Python\Desarrollos\Storm_Detector\GOES_1